In [1]:
# load the libraries that we'll use  
from mutagen.mp3 import MP3  
from mutagen.easyid3 import EasyID3  
# import mutagen.id3  
# from mutagen.id3 import ID3, TIT2, TIT3, TALB, TPE1, TRCK, TYER 
import glob  
import numpy as np 
import os
import json
import csv

In [7]:
class TagEditor:
    album_path = ""
    album_list = []
    
    def __init__(self,album_path,genre,org):
        self.album_path = album_path
        self.genre = genre
        self.org = org
    
    def updateAlbumList(self):
        with open("data.csv", "r") as file:
            reader = csv.reader(file)
            for row in reader:
                self.album_list.append(row)
    
    def renameFolder(self,old,newName):
        new = old.split("\\")
        new[-1] = newName
        os.rename(old,"\\".join(new))
      
    def getAlbums(self):
        count = 0
        for album in os.listdir(self.album_path):
            glob_filez = glob.glob(self.album_path + album + "\\*.mp3")
            self.mp3TagEditor(glob_filez,count)
            album_abs_path = os.path.abspath(os.path.join(album_path, album))
            self.renameFolder(album_abs_path,self.album_list[count][0])
            count += 1
        
    def mp3TagEditor(self,glob_filez, count):
        for i in np.arange(0, len (glob_filez)):
            song = glob_filez[i]  
            # turn it into an mp3 object using the mutagen library  
            mp3file = MP3(song, ID3=EasyID3) 
            
            mp3file['organization'] = [self.org]
            mp3file['genre'] = self.genre
            mp3file['tracknumber'] =  str(i+1) + '/' + str(len(glob_filez))  
            
            mp3file['album'] = [self.album_list[count][0]]
            mp3file['albumartist'] = mp3file['composer'] = [self.album_list[count][1]]
            mp3file['date'] = [self.album_list[count][-1]] 
            
            # mp3file['title'] = [glob_filez[i].split("\\")[-1]]
            # mp3file['artist'] = ['Pawan Kalyan, Trisha Krishnan, Kruthi Kharbanda']
            
            # mp3file['encodedby'] = ['']
            # mp3file['copyright'] = ['http://iSongs.info']
            # save the changes that we've made
            mp3file.save()
            # os.rename(glob_filez[i],glob_filez[i].replace("-SenSongsMp3.Co",""))

In [8]:
# driver code 
if __name__ == '__main__':
    # extract the file names (including folders)  
    # for the mp3s in the album  
    album_path = "C:\\Users\\in the presence\\Documents\\MyWork\\mp3-tag-editor\\test ablums\\"
    genre = "Telugu"
    org = "DSA"
    tagObj = TagEditor(album_path,genre,org)
    tagObj.updateAlbumList()  
    tagObj.getAlbums()  




In [9]:
album_path = "C:\\Users\\in the presence\\Documents\\MyWork\\mp3-tag-editor\\test ablums\\"
for album in os.listdir(album_path):
    glob_filez = glob.glob(album_path+album+"\\*.mp3")
    break

In [10]:

 # print the first element of filez:  
# filez[1]  
print(*glob_filez, sep="\n")

C:\Users\in the presence\Documents\MyWork\mp3-tag-editor\test ablums\1-Nenokkadine (2013)\01 - Who R U.mp3
C:\Users\in the presence\Documents\MyWork\mp3-tag-editor\test ablums\1-Nenokkadine (2013)\02 - Aww Tuzo Mogh Kortha.mp3
C:\Users\in the presence\Documents\MyWork\mp3-tag-editor\test ablums\1-Nenokkadine (2013)\03 - O Sayonara Sayonara.mp3
C:\Users\in the presence\Documents\MyWork\mp3-tag-editor\test ablums\1-Nenokkadine (2013)\04 - You're My Love.mp3
C:\Users\in the presence\Documents\MyWork\mp3-tag-editor\test ablums\1-Nenokkadine (2013)\05 - London Babu.mp3


In [11]:
# we want to extract the track number from the file name  
# the file name is the last part of the string in 'filez'  
# the different parts of 'filez' are split with a   
# forward slash (/)  
# let's split this element up and see what we get  
glob_filez[0].split("\\")

['C:',
 'Users',
 'in the presence',
 'Documents',
 'MyWork',
 'mp3-tag-editor',
 'test ablums',
 '1-Nenokkadine (2013)',
 '01 - Who R U.mp3']

In [29]:
# check the ID3 tags for the mp3s in our folder  
for i in np.arange(0, len(glob_filez)):  
	song = glob_filez[i]  
	mp3file = MP3(song, ID3=EasyID3)  
	print(mp3file["album"][0])
	print(any(x in mp3file["album"][0].split(" ") for x in album_list))
	print(glob_filez[i].split("\\")[-1])
	break

1 - Nenokkadine (2014)
False
01 - Who R U.mp3


In [28]:

def updatemp3tags(mp3file, data):
    try:
        for key, value in data.items():
            if mp3file[key] and mp3file[key][0] not in value:
                value.append(mp3file[key][0])
                data[key] = value
    except Exception as e:
        print("In exeption")
        print(e)   
    return data
    

In [ ]:
# Load the JSON file
with open("mp3tags.json", "r") as file:
    data = json.load(file)
            
# test_album_path = "C:\\Users\\in the presence\\Documents\\MyWork\\mp3-tag-editor\\test ablums\\"
test_album_path = "C:\\Users\\kOUNDINYA\\Music\\Aditya\\Telugu\\"
for album in os.listdir(test_album_path):
    glob_filez = glob.glob(test_album_path+album+"\\*.mp3")
    for i in np.arange(0, len(glob_filez)):  
        song = glob_filez[i]  
        mp3file = MP3(song, ID3=EasyID3) 
        try:
            data = updatemp3tags(mp3file,data)
        except Exception as e:
            pass
            # print("In exeption")
            # print(e)
    
# Write the modified data back to the file
with open("mp3tags.json", "w") as file:
    json.dump(data, file, indent=4)


In [6]:
# test_album_path = "C:\\Users\\kOUNDINYA\\Music\\Aditya\\Telugu\\"
test_album_path = "C:\\Users\\in the presence\\Documents\\MyWork\\mp3-tag-editor\\test ablums"

result = []
for album in os.listdir(test_album_path):
    result.append(album)

with open("output.txt", "w") as file:
    output = "\n".join(result)
    file.write(output)

In [62]:
old = "C:\\Users\\in the presence\\Documents\\MyWork\\mp3-tag-editor\\test ablums"
new = old.split("\\")
new[-1] = "test ablums"
print()
os.rename(old,"\\".join(new))

In [67]:
album_path = "C:\\Users\\in the presence\\Documents\\MyWork\\mp3-tag-editor\\test ablums"
for album in os.listdir(album_path):
    album_abs_path = os.path.abspath(os.path.join(album_path, album))
    print(album_abs_path)

C:\Users\in the presence\Documents\MyWork\mp3-tag-editor\test ablums\1-Nenokkadine (2013)
C:\Users\in the presence\Documents\MyWork\mp3-tag-editor\test ablums\100% Love (2011)
C:\Users\in the presence\Documents\MyWork\mp3-tag-editor\test ablums\118
C:\Users\in the presence\Documents\MyWork\mp3-tag-editor\test ablums\2.0 (2018)
C:\Users\in the presence\Documents\MyWork\mp3-tag-editor\test ablums\24 - (2016)
C:\Users\in the presence\Documents\MyWork\mp3-tag-editor\test ablums\3 Movie 2012
C:\Users\in the presence\Documents\MyWork\mp3-tag-editor\test ablums\7th Sense
C:\Users\in the presence\Documents\MyWork\mp3-tag-editor\test ablums\A Aa (2016)
C:\Users\in the presence\Documents\MyWork\mp3-tag-editor\test ablums\A1 Express (2021)
C:\Users\in the presence\Documents\MyWork\mp3-tag-editor\test ablums\Aacharya
C:\Users\in the presence\Documents\MyWork\mp3-tag-editor\test ablums\Aagadu (2014)
C:\Users\in the presence\Documents\MyWork\mp3-tag-editor\test ablums\aakasame ne haddura
C:\Users\in